In [118]:
import torch
import codecs
import os
import pandas as pd
import numpy as np
import csv
import unicodedata
import re
import itertools
import random

In [3]:
!pip install convokit

  Using cached clean_text-0.6.0-py3-none-any.whl (11 kB)
  Created wheel for convokit: filename=convokit-2.5.3-py3-none-any.whl size=204116 sha256=4d8122da30cbe4e750b25a1552647cc23aa150738fa60d2356ee88459ad668fc
  Stored in directory: c:\users\gowrich1\appdata\local\pip\cache\wheels\b0\e8\2d\81c4477fe586fe4dad2de2886b990e90e839ffccd5158ed0f3
Successfully built convokit


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
from convokit import Corpus, download
corpus = Corpus(filename=download("movie-corpus"))

KeyboardInterrupt: 

### Data Preprocessing

In [3]:
lines_filepath = 'C:/Users/GOWRICH1/Downloads/Learning/Chatbot/cornell movie-dialogs corpus/movie_lines.txt'
conv_filepath = 'C:/Users/GOWRICH1/Downloads/Learning/Chatbot/cornell movie-dialogs corpus/movie_conversations.txt'

In [4]:
with open(lines_filepath, 'r') as file:
    lines = file.readlines()
for line in lines[:8]:
    print(line.strip())

L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!
L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!
L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.
L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?
L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.
L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow
L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.
L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No


In [5]:
line_fields = ["lineID","characterID","movieID","character","text"]
lines={}

with open(lines_filepath, 'r') as f:
    for line in f:
        values = line.split(" +++$+++ ")
        
        lineobj={}
        for i, field in enumerate(line_fields):
            lineobj[field] = values[i]
        lines[lineobj['lineID']] = lineobj

In [6]:
lines

{'L1045': {'lineID': 'L1045',
  'characterID': 'u0',
  'movieID': 'm0',
  'character': 'BIANCA',
  'text': 'They do not!\n'},
 'L1044': {'lineID': 'L1044',
  'characterID': 'u2',
  'movieID': 'm0',
  'character': 'CAMERON',
  'text': 'They do to!\n'},
 'L985': {'lineID': 'L985',
  'characterID': 'u0',
  'movieID': 'm0',
  'character': 'BIANCA',
  'text': 'I hope so.\n'},
 'L984': {'lineID': 'L984',
  'characterID': 'u2',
  'movieID': 'm0',
  'character': 'CAMERON',
  'text': 'She okay?\n'},
 'L925': {'lineID': 'L925',
  'characterID': 'u0',
  'movieID': 'm0',
  'character': 'BIANCA',
  'text': "Let's go.\n"},
 'L924': {'lineID': 'L924',
  'characterID': 'u2',
  'movieID': 'm0',
  'character': 'CAMERON',
  'text': 'Wow\n'},
 'L872': {'lineID': 'L872',
  'characterID': 'u0',
  'movieID': 'm0',
  'character': 'BIANCA',
  'text': "Okay -- you're gonna need to learn how to lie.\n"},
 'L871': {'lineID': 'L871',
  'characterID': 'u2',
  'movieID': 'm0',
  'character': 'CAMERON',
  'text': 'No

In [7]:
conv_fields = ["character1ID","character2ID","movieID","utteranceIDs"]
conversations= []

with open(conv_filepath, 'r') as f:
    for line in f:
        values = line.split(" +++$+++ ")
        # Extract fields
        convobj={}
        for i, field in enumerate(conv_fields):
            convobj[field] = values[i]
        # Convert string from list
        lineIds = eval(convobj['utteranceIDs'])
        # Reassemble lines
        convobj["lines"] = []        
        for lineId in lineIds:
            convobj["lines"].append(lines[lineId])        
        conversations.append(convobj)

In [8]:
conversations[0]

{'character1ID': 'u0',
 'character2ID': 'u2',
 'movieID': 'm0',
 'utteranceIDs': "['L194', 'L195', 'L196', 'L197']\n",
 'lines': [{'lineID': 'L194',
   'characterID': 'u0',
   'movieID': 'm0',
   'character': 'BIANCA',
   'text': 'Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\n'},
  {'lineID': 'L195',
   'characterID': 'u2',
   'movieID': 'm0',
   'character': 'CAMERON',
   'text': "Well, I thought we'd start with pronunciation, if that's okay with you.\n"},
  {'lineID': 'L196',
   'characterID': 'u0',
   'movieID': 'm0',
   'character': 'BIANCA',
   'text': 'Not the hacking and gagging and spitting part.  Please.\n'},
  {'lineID': 'L197',
   'characterID': 'u2',
   'movieID': 'm0',
   'character': 'CAMERON',
   'text': "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"}]}

In [9]:
#Extract pairs of sentences from conversations
qa_pairs=[]

for conversation in conversations:
    
    for i in range(len(conversation['lines'])-1):
        inputline = conversation['lines'][i]['text'].strip()
        targetline = conversation['lines'][i+1]['text'].strip()
        
        if inputline and targetline:
            qa_pairs.append([inputline,targetline])

In [10]:
qa_pairs

[['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.',
  "Well, I thought we'd start with pronunciation, if that's okay with you."],
 ["Well, I thought we'd start with pronunciation, if that's okay with you.",
  'Not the hacking and gagging and spitting part.  Please.'],
 ['Not the hacking and gagging and spitting part.  Please.',
  "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?"],
 ["You're asking me out.  That's so cute. What's your name again?",
  'Forget it.'],
 ["No, no, it's my fault -- we didn't have a proper introduction ---",
  'Cameron.'],
 ['Cameron.',
  "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does."],
 ["The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
  'Seems like she could get a date easy enough...'],
 [

In [16]:
# Define path for new file to write qa pairs
datafile = os.path.join('C:/Users/GOWRICH1/Downloads/Learning/Chatbot/cornell movie-dialogs corpus','formatted_movie_lines.txt')
delimiter = '\t'
#Unescape the delimiter
delimiter = str(codecs.decode(delimiter,"unicode_escape"))

#Write new csv file
print("\nWriting newly formatted file...")
with open(datafile,'w',encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile,delimiter=delimiter)
    for pair in qa_pairs:
        writer.writerow(pair)
print("Done writing to file")


Writing newly formatted file...
Done writing to file


In [17]:
datafile = os.path.join('C:/Users/GOWRICH1/Downloads/Learning/Chatbot/cornell movie-dialogs corpus','formatted_movie_lines.txt')
with open(datafile,'rb') as file:
    lines = file.readlines()
for line in lines[:8]:
    print(line)

b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\r\r\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\r\r\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\r\r\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\r\r\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\r\r\n"
b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\r\r\n"
b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\tSeems like she could get a date easy enough...\r\r\n"
b'Why?\tU

In [57]:
PAD_token = 0 #used for padding short sentences
SOS_token = 1 #start of sentence token <START>
EOS_token = 2 #end of sentence token <END>

class Vocabulary:
    def __init__(self,name):
        self.name = name
        self.word2index = {} #map word to unique index
        self.word2count = {} #counts the frequency of word
        self.index2word = {PAD_token:'PAD',SOS_token:'SOS',EOS_token:'EOS'}
        self.num_words = 3 #Count SOS,EOS,PAD
        
    def addSentence(self,sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    def addWord(self,word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1            
        else:
            self.word2count[word] += 1
    
    # Remove words below a certin count threshold
    def trim(self,min_count):
        keep_words=[]
        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)
                
        print('keep_words {} / {} ={:.4f}'.format(len(keep_words),len(self.word2index),len(keep_words)/len(self.word2index)))
        # Reintialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token:'PAD',SOS_token:'SOS',EOS_token:'EOS'}
        self.num_words = 3 #Count default tokens
        
        for word in keep_words:
            self.addWord(word)        

In [19]:
# Unicode string to plain ASCII
def unicodetoascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD',s) if unicodedata.category(c) != 'Mn') #NFD - Normal Form Decompose(It decomposes the character and combines alltogether)

In [33]:
def normalizestring(s):
    s = unicodetoascii(s.lower().strip())
    s = re.sub(r"([.!?])",r" \1",s) # r to escape a backslash '!' replaced by ' !' substitue character inside [] with whitespace and character
    s = re.sub(r"[^a-zA-Z.!?]+",r" ",s)
    s = re.sub(r"\s+",r" ",s).strip() #Remove multiple spaces 
    return s

In [34]:
normalizestring("aa123aa!s's   dd?")

'aa aa !s s dd ?'

In [59]:
datafile = os.path.join('C:/Users/GOWRICH1/Downloads/Learning/Chatbot/cornell movie-dialogs corpus','formatted_movie_lines.txt')
#Read the file
lines = open(datafile, encoding='utf-8').read().strip().split('\n')
#Split lines into pairs and normalize
pairs =[[normalizestring(s) for s in pair.split('\t')] for pair in lines]
voc = Vocabulary('C:/Users/GOWRICH1/Downloads/Learning/Chatbot/cornell movie-dialogs corpus')

In [60]:
pairs

[['can we make this quick ? roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad . again .',
  'well i thought we d start with pronunciation if that s okay with you .'],
 [''],
 ['well i thought we d start with pronunciation if that s okay with you .',
  'not the hacking and gagging and spitting part . please .'],
 [''],
 ['not the hacking and gagging and spitting part . please .',
  'okay . . . then how bout we try out some french cuisine . saturday ? night ?'],
 [''],
 ['you re asking me out . that s so cute . what s your name again ?',
  'forget it .'],
 [''],
 ['no no it s my fault we didn t have a proper introduction', 'cameron .'],
 [''],
 ['cameron .',
  'the thing is cameron i m at the mercy of a particularly hideous breed of loser . my sister . i can t date until she does .'],
 [''],
 ['the thing is cameron i m at the mercy of a particularly hideous breed of loser . my sister . i can t date until she does .',
  'seems like she coul

In [61]:
MAX_length = 10 #Max sentence words
def filter_pair(p):
    return len(p[0].split())<MAX_length and len(p[1].split())<MAX_length

#Filter pairs using filter_pair
def filter_pairs(pairs):
    return [pair for pair in pairs if filter_pair(pair)]

In [62]:
pairs = [pair for pair in pairs if len(pair)>1]
print('Total pairs/conversations in dataset {}'.format(len(pairs)))
pairs = filter_pairs(pairs)
print('Total pairs/conversations in dataset after filtering {}'.format(len(pairs)))

Total pairs/conversations in dataset 221282
Total pairs/conversations in dataset after filtering 64271


In [63]:
pairs

[['there .', 'where ?'],
 ['you have my word . as a gentleman', 'you re sweet .'],
 ['hi .', 'looks like things worked out tonight huh ?'],
 ['you know chastity ?', 'i believe we share an art instructor'],
 ['have fun tonight ?', 'tons'],
 ['well no . . .', 'then that s all you had to say .'],
 ['then that s all you had to say .', 'but'],
 ['but', 'you always been this selfish ?'],
 ['do you listen to this crap ?', 'what crap ?'],
 ['what good stuff ?', 'the real you .'],
 ['the real you .', 'like my fear of wearing pastels ?'],
 ['wow', 'let s go .'],
 ['she okay ?', 'i hope so .'],
 ['they do to !', 'they do not !'],
 ['did you change your hair ?', 'no .'],
 ['no .', 'you might wanna think about it'],
 ['who ?', 'joey .'],
 ['great', 'would you mind getting me a drink cameron ?'],
 ['it s more', 'expensive ?'],
 ['hey sweet cheeks .', 'hi joey .'],
 ['where ve you been ?', 'nowhere . . . hi daddy .'],
 ['you are so completely unbalanced .', 'can we go now ?'],
 ['what ?', 'in th . fo

In [64]:
for pair in pairs:
    voc.addSentence(pair[0])
    voc.addSentence(pair[1])
print("Counted words: ", voc.num_words)

for pair in pairs[:10]:
    print(pair)

Counted words:  18008
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']


In [65]:
Min_count = 3

def trimRareWords(voc, pairs, Min_count):
    #Trim words used under Min_count from voc
    voc.trim(Min_count)
    #Filter out pairs with trimmed words
    keep_pairs=[]
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        #Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        #Check Output Sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break
        
        #only keep pairs that do not contain trimmed words in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)
            
    print('Trimmed from {} pairs to {},{:.4f}  of total'.format(len(pairs),len(keep_pairs),len(keep_pairs)/len(pairs)))
    return keep_pairs

#Trim voc and pairs
pairs = trimRareWords(voc, pairs, Min_count)

keep_words 7823 / 18005 =0.4345
Trimmed from 64271 pairs to 53165,0.8272  of total


In [69]:
def indexesfromsentence(voc,sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

In [73]:
pairs[110][1]

'no i won t go with you'

In [72]:
indexesfromsentence(voc,pairs[110][1])

[34, 25, 331, 117, 60, 169, 7, 2]

In [78]:
#Define some samples for testing
inp = []
out = []
for pair in pairs[:10]:
    inp.append(pair[0])
    out.append(pair[1])
print(inp)
print(len(inp))
indexes = [indexesfromsentence(voc,sentence) for sentence in inp]
indexes

['there .', 'you have my word . as a gentleman', 'hi .', 'have fun tonight ?', 'well no . . .', 'then that s all you had to say .', 'but', 'do you listen to this crap ?', 'what good stuff ?', 'wow']
10


[[3, 4, 2],
 [7, 8, 9, 10, 4, 11, 12, 13, 2],
 [16, 4, 2],
 [8, 31, 22, 6, 2],
 [33, 34, 4, 4, 4, 2],
 [35, 36, 37, 38, 7, 39, 40, 41, 4, 2],
 [42, 2],
 [47, 7, 48, 40, 45, 49, 6, 2],
 [50, 51, 52, 6, 2],
 [58, 2]]

In [99]:
a = [1,2,3,4,5]
b = [1,2,3]
print(list(itertools.zip_longest(a,b)))
print(list(zip(a,b)))

[(1, 1), (2, 2), (3, 3), (4, None), (5, None)]
[(1, 1), (2, 2), (3, 3)]


In [75]:
def zeropadding(l, fillvalue=0):
    return list(itertools.zip_longest(*l,fillvalue=fillvalue))

In [79]:
leng = [len(ind) for ind in indexes]
max(leng)

10

In [80]:
#Test the function
test_result = zeropadding(indexes)
print(len(test_result)) # The max length is now the number of rows or the batch size
test_result

10


[(3, 7, 16, 8, 33, 35, 42, 47, 50, 58),
 (4, 8, 4, 31, 34, 36, 2, 7, 51, 2),
 (2, 9, 2, 22, 4, 37, 0, 48, 52, 0),
 (0, 10, 0, 6, 4, 38, 0, 40, 6, 0),
 (0, 4, 0, 2, 4, 7, 0, 45, 2, 0),
 (0, 11, 0, 0, 2, 39, 0, 49, 0, 0),
 (0, 12, 0, 0, 0, 40, 0, 6, 0, 0),
 (0, 13, 0, 0, 0, 41, 0, 2, 0, 0),
 (0, 2, 0, 0, 0, 4, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 2, 0, 0, 0, 0)]

In [120]:
def binarymatrix(l,value=0):
    m=[]
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

In [105]:
binarymatrix = binarymatrix(test_result)
binarymatrix

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 1, 1, 0],
 [0, 1, 0, 1, 1, 1, 0, 1, 1, 0],
 [0, 1, 0, 1, 1, 1, 0, 1, 1, 0],
 [0, 1, 0, 0, 1, 1, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]]

In [134]:
# Returns padded input sequence tensor and as well as a tensor of lengths for each of the sequences of the batch
def inputvar(l,voc):
    print(l)
    indexes_batch = [indexesfromsentence(voc,sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padlist = zeropadding(indexes_batch)
    padvar = torch.LongTensor(padlist)
    return padvar, lengths

In [135]:
# Returns padded target sequence tensor, padding mask, and max target length
def outputvar(l,voc):
    print(l)
    indexes_batch = [indexesfromsentence(voc,sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padlist = zeropadding(indexes_batch)
    mask = binarymatrix(padlist)
    mask = torch.ByteTensor(mask)
    padvar = torch.LongTensor(padlist)
    return padvar, mask, max_target_len

In [136]:
# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    #sort questions in descending length
    pair_batch.sort(key=lambda x: len(x[0].split(" ")),reverse = True)
    input_batch, output_batch = [],[]
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputvar(input_batch,voc)
    output, mask, max_target_len = outputvar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len

In [137]:
# Validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for i in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

['you re out of your mind .', 'beautiful view isn t it ?', 'who we gonna beat ?', 'to do business .', 'where we going ?']
['he s an enemy soldier .', 'walter there s something i have to tell you', 'who we gonna beat ?', 'i can help you .', 'quick drop . in and out .']


In [138]:
input_variable

tensor([[   7,  107,   77,   40,    5],
        [  14, 5071,   27,   47,   27],
        [  21,  271,  509,  803,  123],
        [  56,  117, 2000,    4,    6],
        [  70,   76,    6,    2,    2],
        [  81,    6,    2,    0,    0],
        [   4,    2,    0,    0,    0],
        [   2,    0,    0,    0,    0]])

In [139]:
lengths

tensor([8, 7, 6, 5, 5])

In [140]:
target_variable

tensor([[ 101, 1134,   77,   25, 3249],
        [  37,    3,   27,   94, 1954],
        [  29,   37,  509,  411,    4],
        [3473,  319, 2000,    7,   96],
        [1621,   25,    6,    4,  122],
        [   4,    8,    2,    2,   21],
        [   2,   40,    0,    0,    4],
        [   0,  141,    0,    0,    2],
        [   0,    7,    0,    0,    0],
        [   0,    2,    0,    0,    0]])

In [141]:
mask

tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0]], dtype=torch.uint8)